In [1]:
import yaml
import wandb
import torchreid

config_path = "configs/im_osnet_ain_x1_0_softmax_256x128_amsgrad_cosine.yaml"
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

wandb.init(project=config['wandb']['project'],
           entity=config['wandb']['entity'],
           config=config)


/home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: balamanikandan (tangoeye). Use `wandb login --relogin` to force relogin


In [2]:
import torchreid
datamanager = torchreid.data.ImageDataManager(
    root            = config['data']['root'],
    sources         = config['data']['sources'],
    targets         = config['data']['targets'],
    height          = config['data']['height'],
    width           = config['data']['width'],
    batch_size_train= config['data']['batch_size'],
    batch_size_test = config['data']['batch_size'],
    transforms      = config['data']['transforms']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ color jitter
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6


In [3]:
model = torchreid.models.build_model(
    name        = config['model']['name'],
    num_classes = 2000 if 'tango2000' in config['data']['sources'] else datamanager.num_train_pids, #2000 for Tango
    loss        = config['loss']['name'],
    pretrained  = config['model']['pretrained'],
    pooling     = config['model']['final_pooling'],
    **config['custom']
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [4]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [5]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
    logger=wandb
)

In [6]:
engine.run(
    save_dir=f"log/{config['model']['name']}_{config['data']['sources'][0]}",
    max_epoch=60,
    eval_freq=5,
    print_freq=50,
    test_only=False,
    visrank=False,
    eval_metric='default'
)

=> Start training
epoch: [1/60][50/101]	time 0.151 (0.169)	data 0.000 (0.006)	eta 0:16:55	loss 5.5082 (6.2888)	acc 9.3750 (5.4219)	lr 0.000300
epoch: [1/60][100/101]	time 0.149 (0.159)	data 0.000 (0.003)	eta 0:15:49	loss 3.9327 (5.3938)	acc 31.2500 (15.2891)	lr 0.000300
epoch: [2/60][50/101]	time 0.150 (0.154)	data 0.000 (0.004)	eta 0:15:12	loss 2.5561 (2.9104)	acc 68.7500 (55.8594)	lr 0.000300
epoch: [2/60][100/101]	time 0.154 (0.153)	data 0.000 (0.002)	eta 0:14:57	loss 1.9116 (2.5736)	acc 77.3438 (63.7656)	lr 0.000300
epoch: [3/60][50/101]	time 0.147 (0.151)	data 0.001 (0.004)	eta 0:14:37	loss 1.6080 (1.7010)	acc 87.5000 (87.0000)	lr 0.000300
epoch: [3/60][100/101]	time 0.155 (0.150)	data 0.000 (0.002)	eta 0:14:21	loss 1.5150 (1.6519)	acc 89.0625 (87.8125)	lr 0.000300
epoch: [4/60][50/101]	time 0.147 (0.152)	data 0.000 (0.004)	eta 0:14:27	loss 1.3871 (1.4088)	acc 93.7500 (94.0156)	lr 0.000300
epoch: [4/60][100/101]	time 0.147 (0.150)	data 0.000 (0.002)	eta 0:14:09	loss 1.4656 (1.3991